In [1]:
import pandas as pd
import math

In [2]:
class Tree:
    variable = None
    entropy = 0
    YES = None
    NO = None
    def __init__(self,c):
        self.variable = c
def printTree(root):
    print(root.variable)
    
    if(root.YES):
        printTree(root.YES)
    if(root.NO):
        printTree(root.NO)
class ID3:
    def __init__(self, data):
        self.TreeRoot = self.GetNode(data)
    def getParentEntropy(self, data):
        y = 0
        for i in data:
            y+=i
        n = data.shape[0] - y
        if not (n and y):
            return 0
        n = n/(n+y)
        y = y/(n+y)
        return (-1 * n* math.log2(n)) + (-1 * y * math.log2(y))
    def getChildEntropy(self, data, target):
        a = self.getMatrix(data,target)
        p,q,r,s = a[0][0], a[0][1], a[1][0], a[1][1] 
        
        A = p+q
        B = r + s
        A = A/(A+B)
        B = B/(A+B)
        n = 0
        y = 0
        if(p+q==0):
            n = 0
        else:
            p = p/(p+q)
            q = q/(p+q)
            n=0
            if(p!=0):
                n = (-1 * p* math.log2(p))
            if(q!=0):
                n+= (-1 * q * math.log2(q))
            n = A*n
        if(r+s==0):
            y = 0
        else:
            r = r/(s+r)
            s = s/(s+r)
            y = 0
            if(r!=0):
                y+=(-1 * r* math.log2(r))
            if(s!=0):
                y+= (-1 * s * math.log2(s))
            y = B*n
        return n+y     
    def getTable(self, data, colum, value):
        result = []
        for i in range(0,data.shape[0]):
            if(colum.iloc[i] == value):
                result.append(data.iloc[i])
        return pd.DataFrame(result)
    def getMatrix(self, column, target):
        matri = [[0,0],[0,0]]
        #print(column)
        for i in range(0,column.shape[0]):
            matri[column.iloc[i]][target.iloc[i]]+=1
        return matri
    def GetNode(self,data):
        if(data.shape[0]==0):
            return Tree("YES") 
        col = list(data.columns)
        if(data.shape[1]>2):
            Gain = []
            target = data[col.pop()]
            ParentEntropy = self.getParentEntropy(target)
            if(ParentEntropy==0):
                if(target.iloc[0]==0):
                    return Tree("NO")
                else:
                    return Tree("YES")
            print(ParentEntropy)
            for i in col:
                ChildEntropy = self.getChildEntropy(data[i],target)
                InformationGain = ParentEntropy - ChildEntropy
                Gain.append([InformationGain,i])
            print(Gain)
            Gain.sort(reverse = True)
            root = Tree(Gain[0][1])
            root.entropy = Gain[0][0]
            rootColumn = data[Gain[0][1]]
            #print(rootColumn.shape, len(rootColumn.shape))
            print(Gain[0][1])
            data.drop(str(Gain[0][1]), axis=1, inplace=True)
#             print(type(data))
#             print(data.shape)
            Y = self.getTable(data, rootColumn, 1)
            print(Y)
            N = self.getTable(data, rootColumn, 0)
            print(N)
            print(Y.shape,N.shape, root.variable)
            if(Y.shape[0]==0):
                root.YES = Tree("YES")
            else:
                root.YES = self.GetNode(Y)
            if(N.shape[0]==0):
                root.NO = Tree("NO")
            else:
                root.NO = self.GetNode(N)
            return root
        
        root = Tree(col[0])
        Target = data[col[1]]
        data.drop(col[1],axis = 1, inplace = True)
        M = self.getMatrix(data[col[0]], Target)
        if(M[0][0]+M[1][1]>=M[0][1]+M[1][0]):
            root.YES = Tree("YES")
            root.NO = Tree("NO")
        else:
            root.YES = Tree("NO")
            root.NO = Tree("YES")
        return root

In [3]:
data = {
    'A': [1, 1, 1, 0, 0, 0, 0, 0],
    'B': [1, 0, 1, 0, 0, 1, 0, 0],
    'C': [0, 0, 1, 0, 1, 1, 1, 0],
    'D': [0, 0, 1, 0, 1, 1, 1, 1],
    'E': [1, 1, 1, 0, 0, 1, 0, 0]}
data = pd.DataFrame(data)


In [4]:
Tr = ID3(data)

0.6510444457264999
[[-0.1812518695179426, 'A'], [-0.1812518695179426, 'B'], [-0.064412337077285, 'C'], [0.13064681682465396, 'D']]
D
   A  B  C  E
2  1  1  1  1
4  0  0  1  0
5  0  1  1  1
6  0  0  1  0
7  0  0  0  0
   A  B  C  E
0  1  1  0  1
1  1  0  0  1
3  0  0  0  0
(5, 4) (3, 4) D
0.7333421436179774
[[-0.22814524472768416, 'A'], [0.7333421436179774, 'B'], [0.7333421436179774, 'C']]
C
   A  B  E
2  1  1  1
4  0  0  0
5  0  1  1
6  0  0  0
   A  B  E
7  0  0  0
(4, 3) (1, 3) C
0.7575424759098899
[[-0.22321087338855006, 'A'], [0.7575424759098899, 'B']]
B
   A  E
2  1  1
5  0  1
   A  E
4  0  0
6  0  0
(2, 2) (2, 2) B
0.7189429090049599
[[0.7189429090049599, 'A'], [-0.23036375817452903, 'B'], [0.0, 'C']]
A
   B  C  E
0  1  0  1
1  0  0  1
   B  C  E
3  0  0  0
(2, 3) (1, 3) A


In [5]:
printTree(Tr.TreeRoot)

D
C
B
A
YES
NO
A
YES
NO
NO
A
YES
NO
